<a href="https://colab.research.google.com/github/rake220/my-first/blob/main/TextGen_by_GPT2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets torch accelerate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 118.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 89.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments
from datasets import load_dataset


In [3]:
model_name = "distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [4]:
tokenizer.pad_token = tokenizer.eos_token


In [5]:
dataset = load_dataset("wikitext", "wikitext-2-raw-v1", split="train[:10%]")


README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/733k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/6.36M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/657k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

In [6]:
def tokenize_function(examples):
    tokens = tokenizer(examples["text"], padding="max_length", truncation=True, max_length=64)
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens


In [7]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/3672 [00:00<?, ? examples/s]

In [8]:

dataset = tokenized_datasets.train_test_split(test_size=0.1)
train_dataset = dataset["train"]
eval_dataset = dataset["test"]



In [9]:
training_args = TrainingArguments(
    output_dir="./gpt2-finetuned-Rak",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
    fp16=True,
    gradient_accumulation_steps=8,
    remove_unused_columns=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [10]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: rgurram09 (rgurram09-stevens-institute-of-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Epoch,Training Loss,Validation Loss
1,No log,1.405054
2,1.844800,1.357162


TrainOutput(global_step=618, training_loss=1.7961639108009708, metrics={'train_runtime': 262.2494, 'train_samples_per_second': 37.796, 'train_steps_per_second': 2.357, 'total_flos': 161220095115264.0, 'train_loss': 1.7961639108009708, 'epoch': 2.9878934624697338})

In [11]:
eval_results = trainer.evaluate()
print(f"Evaluation Results: {eval_results}")


Evaluation Results: {'eval_loss': 1.357162356376648, 'eval_runtime': 1.784, 'eval_samples_per_second': 206.281, 'eval_steps_per_second': 103.141, 'epoch': 2.9878934624697338}


In [12]:
def generate_text(prompt):
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(model.device)
    output = model.generate(
        input_ids=input_ids,
        num_beams=5,
        max_length=200,
        no_repeat_ngram_size=2,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id
    )
    return tokenizer.decode(output[0], skip_special_tokens=True)


In [13]:
prompt = "Once upon a time in a distant kingdom,"
generated_text = generate_text(prompt)
print(f"Generated Text: {generated_text}")



The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Generated Text: Once upon a time in a distant kingdom, the king of the Kingdom of Cairns , who had been appointed by the King of Wales , was appointed to the throne by King Henry VIII . He was the first to be appointed as a successor to his predecessor , Henry VI , and the second to become the new king . The first of his successors , William of York , appointed the newly appointed king as his successor . In the reign of Henry III , however , he was replaced by his younger brother , Edward III of Cornwall , the son of Edward VI of England . Edward II was succeeded by Henry IV of Great Britain , but was defeated by Richard III in the Battle of Leicestershire , which resulted in his death . His successor , Robert II of France , had the power to appoint a new governor of that kingdom . Robert III was elected to replace the previous king , Richard II , by a vote of 1 to 1 . After the death of Robert IV in 1783 , his replacement was


In [14]:
model.save_pretrained("./gpt2-finetuned-Rakesh")
tokenizer.save_pretrained("./gpt2-finetuned-Rakesh")

('./gpt2-finetuned-Rakesh/tokenizer_config.json',
 './gpt2-finetuned-Rakesh/special_tokens_map.json',
 './gpt2-finetuned-Rakesh/vocab.json',
 './gpt2-finetuned-Rakesh/merges.txt',
 './gpt2-finetuned-Rakesh/added_tokens.json',
 './gpt2-finetuned-Rakesh/tokenizer.json')

In [15]:
eval_results = trainer.evaluate()
print(f"Evaluation Results: {eval_results}")


Evaluation Results: {'eval_loss': 1.357162356376648, 'eval_runtime': 2.7663, 'eval_samples_per_second': 133.032, 'eval_steps_per_second': 66.516, 'epoch': 2.9878934624697338}


In [16]:
prompt = "Virat Kohli is a Cricketer,"
generated_text = generate_text(prompt)
print(f"Generated Text: {generated_text}")


Generated Text: Virat Kohli is a Cricketer, who has won four of his last five @-@ matches for the first time in his career . He was named to the All India team of the year in the 2014 World Twenty20 , and was the only player to have won the award . In his first match against Sri Lanka , he scored a hat trick against the Sri Lankan side , who had won three of their last four @.@santasil matches . 



# New Section